In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from datetime import datetime

from snobedo.lib.dask_utils import run_with_client

from common import *
from mcs_shared import SNOBAL_DIR, DATA_DIR, FIRST_WATER_YEAR, \
    load_snotel_locations, mcs_snotel_depth

## SNOTEL 

In [ ]:
snotel_sites = load_snotel_locations()
snotel_depth = mcs_snotel_depth(datetime(FIRST_WATER_YEAR - 1, 10, 1), datetime(2024, 7, 31))

## Model

In [ ]:
with run_with_client(10, 32):
    mcs_snobal = xr.open_mfdataset(
        (SNOBAL_DIR / f'wy202[0,1,2,3,4]' / 'mcs/*/snow.nc').as_posix(),
        preprocess=lambda ds: ds['thickness'],
        parallel=True,
    ).sel(
        x=snotel_sites.Mores_Creek_Summit.lon, 
        y=snotel_sites.Mores_Creek_Summit.lat,
        method='nearest',
    ).compute()

# Matplotlib 

In [ ]:
import matplotlib.ticker as mticker
from matplotlib.offsetbox import AnchoredText

In [ ]:
STATION_LABEL = 'Station'
COLORS = {
        'iSnobal': 'cornflowerblue',
        STATION_LABEL: 'seagreen',    
}

def plot_site(data, ax, xlim):
    for key in data:
        if key == STATION_LABEL:
            continue

        ax.plot(
            data[key].time,
            data[key].data.flatten(),
            label=key, 
            color=COLORS[key], 
            alpha=0.9, lw=1
        )
    ax.plot(
        data[STATION_LABEL].index,
        data[STATION_LABEL].values,
        label=key, 
        color=COLORS[STATION_LABEL], 
        alpha=0.9, lw=1.25
    ) 
    
    ax.set_ylabel(r'Snow Depth (m)')

    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)

    ax.set_xlim([xlim[0], xlim[-1]])
    ax.set_yticks(np.arange(0, 4))
    ax.set_ylim(bottom=-0.05)
    ax.set_ylim(top=3)
        
    ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.2))

In [ ]:
periods = 58
plot_range = pd.date_range(start=f'{FIRST_WATER_YEAR - 1}-10-01', periods=periods, freq='MS')
xTicks = mdates.DateFormatter('%b-%Y')

figure_opts = dict(figsize=(6,3), dpi=300,)
fig, axes = plt.subplots(1, 1, sharex=True, **figure_opts)
plt.subplots_adjust(hspace=0.05)

plot_site(
    {
        'iSnobal': mcs_snobal['thickness'],
        STATION_LABEL: snotel_depth,
    },
    axes,
    plot_range
)

axes.tick_params(axis='x', which='major', pad=0, labelrotation=45)

# at = AnchoredText(
#     f'Water Year {year} - 2022', 
#     prop=dict(size=10), 
#     frameon=False, 
#     loc='upper left', 
#     pad=0.3, 
#     borderpad=0.25,
# )
# axes.add_artist(at)
axes.legend(
    frameon=False,
    bbox_to_anchor=(0.4, 0.99),
    ncol=5,
    borderaxespad=0.15, 
    fontsize=8
);

# HV plot

## SNOTEL 

In [ ]:
use_hvplot()

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
y_lim = (-0.05, 3.)
SNOTEL_LINE=dict(line_width=3)

In [ ]:
mcs_snobal.thickness.hvplot(label='iSnobal').opts(
    title='Mores Creek Summit', ylim=y_lim, **LINE_STYLE, **HV_PLOT_OPTS
) * \
snotel_depth.plot(label='Station', **SNOTEL_LINE)